In [1]:
# Aurthor: Elihu Essien-Thomspon
# Program Objective: Creating the
# hybrids for the final expriment

import numpy as np
import random
import math

In [2]:
# Returns the distance between two 
# points(x, y) on the map

def cityDistance(map, cityID1, cityID2):
    x = abs(map[cityID1][0] - map[cityID2][0])
    y = abs(map[cityID1][1] - map[cityID2][1])
    
    # pythagoras
    dist = math.sqrt(x**2 + y**2)
    return(dist)

In [3]:
# Reading in map data from text file
# and returning a distance matrix to use
# for calculations
def getMap(size,id):
    cities = np.empty((0,2), dtype=int)      # initialize empty 2D vector list
    
    # open file
    with open(f"C:/Users/C14460702/Dissertation/Data/Maps/Size - {size}/map{id}.txt", 'r') as f:
        # extract vector list data from the comma separated string list data
        for line in f:
            line = line.strip('\n')    # remove '\n'
            vec = line.split(', ')     # split by ', '
            
            # cast co-ordinate vector as integers and insert into list 
            cities = np.append(cities, [[ int(vec[0]), int(vec[1]) ]], axis=0)
    
    
    # initialize distance table
    mapSize = len(cities)
    distanceTable = np.zeros((mapSize,mapSize))
    for row in range(mapSize):
        for col in range(mapSize):
            distanceTable[row][col] = cityDistance(cities, row, col)
    
    # return distance matrix
    return (distanceTable)

In [4]:
# Function takes in a solution route and a distance matrix to be used
# to score the given route

def routeScore(route, distanceTable):
    dist = 0;
    
    # aggregate distances between cities
    # by following the route sequence
    for i in range(len(route)):
        cur = route[i]
        nxt = route[(i+1) % len(route)]  # includes loop back to start
        dist += distanceTable[cur][nxt]  # get the distances from the matrix
        
    
    # the smaller the distance, the higher the score
    return (1/dist)

In [5]:
# maps a variable exising on one range to another range
def mapRanges(value, currlow, currHigh, newLow, newHigh):
    # extract ranges
    currRange = currHigh - currlow
    newRange = newHigh - newLow
    
    # percentage of range that the value takes up
    valPercent = float(value - currlow) / float(currRange)
    
    # translate percentage to new range
    newVal = (newRange * valPercent) + newLow
    
    return(newVal)

In [6]:
# This function takes in a 1D array of choices
# where the each member's ID represents the choice
# and each member's value represents the percent (0-1)
# that member has of comming true. The algorithm then
# selects a winner by performing a weighted random choice

def weightedChoice(percentages):
    choiceIDs = [i for i in range(len(percentages))]
    percentages = [mapRanges(i, 0, 1, 0, 100) for i in percentages]
    
    # choose winner
    winnerID = random.choices(choiceIDs, weights = percentages, k = 1)[0]
    return (winnerID)


In [7]:
class Particle:
    def __init__(self, route, distanceTable):
        self.route = route
        self.distanceTable = distanceTable
        self.score = routeScore(route, distanceTable)
        self.pBest = [self.route, self.score]
        self.velocity = []  #swap sequinces to transform map state
    
    def updateRecords(self):
        self.score = routeScore(self.route, self.distanceTable)
        if(self.score > self.pBest[1]):
            self.pBest = [self.route, self.score]
            
    def getRoute(self):
        return(self.route)
    
    def setRoute(self, route):
        self.route = route
        self.updateRecords()
    
    def getScore(self):
        return(self.score)
    
    def caculateNewVelocity(self, bestSolutionFound, w):
        # stochastic probabilities for adding swap sequences
        # to the final velocity
        pBestProb = random.random()
        gBestProb = random.random()
        
        # apply inertia
        self.velocity = [i for i in self.velocity if (random.random() < w)]
        
        # calculate swap sequence to get to pBest
        cur = [i for i in self.route] # temporary list used to validate swap sequence logic
        nxt = [i for i in self.pBest[0]] # target route
        for i in range(len(cur)):
            if(cur[i] != nxt[i]):
                j = cur.index(nxt[i])            # position to swap with
                cur[i], cur[j] = cur[j], cur[i]  # swap temp list to keep track
                
                # add current swap to velocity list using pBest probability
                if (random.random() < pBestProb):
                    self.velocity += [[i, j]]
        
        # calculate swap sequence to get to gBest
        cur = [i for i in self.route] # temporary list used to validate swap sequence logic
        nxt = [i for i in bestSolutionFound] # target route
        for i in range(len(cur)):
            if(cur[i] != nxt[i]):
                j = cur.index(nxt[i])            # position to swap with
                cur[i], cur[j] = cur[j], cur[i]  # swap temp list to keep track
                
                # add current swap to velocity list using gBest probability
                if (random.random() < gBestProb):
                    self.velocity += [[i, j]]
        
    def move(self):
        # swap cities using swap sequence
        for swap in self.velocity:
            self.route[swap[0]], self.route[swap[1]] = self.route[swap[1]], self.route[swap[0]]
        
        self.updateRecords()

In [8]:
def Ant(startingPosition, mapSize, distanceTable, pheromoneTable, alpha, beta):
    position = startingPosition
    visited = []
    
    while (len(visited) < mapSize-1):
        # add current city possition to the visited list
        visited += [position]
        
        # next city options
        ChoiceIDs = [i for i in range(mapSize) if i not in visited]

        # calculate choice weights
        choiceAttractiveness = [(pheromoneTable[position][cityID] ** alpha) * ((1/distanceTable[position][cityID]) ** beta) for cityID in ChoiceIDs]
        total = sum(choiceAttractiveness)
        weigths = [i/total for i in choiceAttractiveness]

        # move to next city
        position = ChoiceIDs[ weightedChoice(weigths) ]

    # add the final position to the list
    visited += [position]
    
    # the order in which the cities were visited
    return(visited)

In [9]:
# Pheramone update function for the Ant System (AS)
# for the TSP allows all ants to update the pheramone
# table adding pheramones to thier traveled paths 
# relative to the overall score of thier chosen route
def PheromoneUpdate(routes, scores, pheromoneTable):
    
    
    for i in range(len(routes)):
        route = routes[i]
        
        # update pheramone levels for the paths taken
        # with the overall score of the route
        for j in range(len(route)):
            cur = route[j]
            nxt = route[(j+1) % len(route)]
            
            # update each edge with the score for that route
            pheromoneTable[cur][nxt] += scores[i]
            pheromoneTable[nxt][cur] += scores[i]
    
    return (pheromoneTable)

In [10]:
# conducts algortihm by dividing the population
# in half, each half utilizing a diferent mechanism
# for optimization
def PSO_ACO_Hybrid(mapSize, popSize, maxIterations, distanceTable):
    # Step 1 - Initialization
    # optimal ACO settings drawn from experiment 2
    evaporationRate = 0.9
    alpha = 1
    beta = 2
    
    # optimal PSO settings drawn from experiment 3
    w = 0.4
    alpha = 0.3
    beta = 0.7
    itPerc = 0.5
    tuningProb = alpha
    
    # algorithm settings
    pheromoneTable = [[0.0000000001 for col in range(mapSize)] for row in range(mapSize)]
    defaultRoute = [i for i in range(mapSize)]
    particles = [Particle(random.sample(defaultRoute, mapSize), distanceTable) for i in range(popSize)]
    GBestScore_PerItreation = []
    gBest = [[],0] # [route, score] for all time
    
    for iteration in range(maxIterations):
        # Step 2 - Run PSO
        # get the best solution found in this iteration
        gcBest = [[],0]  # [route, score] per iteration
        for member in particles:
            if(member.getScore() > gcBest[1]):
                gcBest = [member.getRoute(), member.getScore()]
        
        # get the all time best solution found
        if(gcBest[1] > gBest[1]):
                gBest = [i for i in gcBest]
        
        if(random.random() < tuningProb): # use the gbest
            for member in particles:
                member.caculateNewVelocity(gBest[0], w)
                member.move()
        else:                             # use the gcbest
            for member in particles:
                member.caculateNewVelocity(gcBest[0], w)
                member.move()
            if(tuningProb < beta):
                    tuningProb += (alpha-beta)/(popSize * itPerc)
        
        
        
        # Step 3 - Update Pheramone table and records
        routes = []
        scores = []
        for member in particles:
            routes += [member.getRoute()]
            scores += [member.getScore()]
        pheromoneTable = PheromoneUpdate(routes, scores, [i for i in pheromoneTable])
        
        
        
        # Step 4 - run ACO using updated pheramone table
        # recycled the routes and scores table
        for i in range(popSize):
            routes[i] = Ant(random.choice(range(mapSize)), mapSize, distanceTable, pheromoneTable, alpha, beta)
            scores[i] = routeScore(routes[i], distanceTable)
        
        
        
        # Step 5 - Update Pheramone table and records
        pheromoneTable = [[(1-evaporationRate) * col for col in row] for row in pheromoneTable]  # evaporation
        pheromoneTable = PheromoneUpdate(routes, scores, [i for i in pheromoneTable])        # deposit
        
        for i in range(len(routes)):
            if(scores[i] > gcBest[1]):
                gcBest = [routes[i], scores[i]]
                
        if(gcBest[1] > gBest[1]):
                gBest = [i for i in gcBest]
        GBestScore_PerItreation += [gBest[1]]
        
    return(GBestScore_PerItreation)
    

In [28]:
# Experiment 4 collecting data from hybrid to solve
# the final Research Question

popSize = 50            # Number of chromosomes to use
numCities = 50        # cities per map
maxIterations = 200    # Maximum number of generations

# Traveling Salesman Problem Settings
mapCounter = 0
while (True):
    try:
        # get map
        distanceTable = getMap(numCities,mapCounter)
        
        with open (f"C:/Users/C14460702/Dissertation/Data/Results/Experiment 4/Size - {numCities}/PSO_ACO_Hybrid.txt", "a") as f:
            data = PSO_ACO_Hybrid(numCities, popSize, maxIterations, distanceTable)
            f.writelines(str(data))
        
        print(f"Finished map {mapCounter+1}!")
        mapCounter += 1
    except FileNotFoundError:
        break

Finished map 1!
Finished map 2!
Finished map 3!
Finished map 4!
Finished map 5!
Finished map 6!
Finished map 7!
Finished map 8!
Finished map 9!
Finished map 10!
Finished map 11!
Finished map 12!
Finished map 13!
Finished map 14!
Finished map 15!
Finished map 16!
Finished map 17!
Finished map 18!
Finished map 19!
Finished map 20!
Finished map 21!
Finished map 22!
Finished map 23!
Finished map 24!
Finished map 25!
Finished map 26!
Finished map 27!
Finished map 28!
Finished map 29!
Finished map 30!
Finished map 31!
Finished map 32!
Finished map 33!
Finished map 34!
Finished map 35!
Finished map 36!
Finished map 37!
Finished map 38!
Finished map 39!
Finished map 40!
Finished map 41!
Finished map 42!
Finished map 43!
Finished map 44!
Finished map 45!
Finished map 46!
Finished map 47!
Finished map 48!
Finished map 49!
Finished map 50!
Finished map 51!
Finished map 52!
Finished map 53!
Finished map 54!
Finished map 55!
Finished map 56!
Finished map 57!
Finished map 58!
Finished map 59!
Finish